In [1]:
import os
import textwrap
os.environ["CUDA_VISIBLE_DEVICES"] = "0"


In [4]:
from datasets import load_dataset
ds = load_dataset(
    "hwaseem04/Aya-testing",
    data_files={"xGQA_vqa": "data/xGQA_vqa-00000-of-00001.parquet"}
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


xGQA_vqa-00000-of-00001.parquet:   0%|          | 0.00/340M [00:00<?, ?B/s]

Generating xGQA_vqa split: 0 examples [00:00, ? examples/s]

In [5]:
ds['xGQA_vqa'][0]

{'sample_id': 1,
 'image_id': 'n161313',
 'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=500x280>,
 'question_en': 'Is it overcast?',
 'answer_en': 'No, it is clear.',
 'question_bn': 'এটা কি মেঘাচ্ছন্ন?',
 'answer_bn': 'না, এটা স্পষ্ট।',
 'question_de': 'Ist es bewölkt?',
 'answer_de': 'Nein, es ist klar.',
 'question_id': 'Apakah langit mendung ?',
 'answer_id': 'Tidak, itu jelas.',
 'question_ko': '날이 흐린가요?',
 'answer_ko': '아니, 분명해요.',
 'question_ru': 'Это пасмурная погода?',
 'answer_ru': 'Нет, это ясно.',
 'question_zh': '是阴天吗？',
 'answer_zh': '不，很清楚。'}

In [6]:
from huggingface_hub import notebook_login
notebook_login()

In [7]:
from datasets import load_dataset
from tqdm import tqdm
from PIL import Image
import os

import torch
from transformers import AutoProcessor, Gemma3ForConditionalGeneration

# Languages to iterate over
languages = ["en", "bn", "de", "ko", "ru", "zh"]

# Directory to save temp images (needed for this model)
os.makedirs("temp_images_vqa", exist_ok=True)

# Load Gemma model and processor
model_id = "google/gemma-3-12b-it"
model = Gemma3ForConditionalGeneration.from_pretrained(model_id, device_map="auto").eval()
processor = AutoProcessor.from_pretrained(model_id)

# Set torch dtype based on device
torch_dtype = torch.bfloat16 if torch.cuda.is_available() else torch.float32

dataset = ds['xGQA_vqa']

for sample in tqdm(dataset, desc="Iterating samples"):
    try:
        image = sample["image"]
        sample_id = sample["sample_id"]

        #### This part will be replaced when attack implementation is ready #####

        image_path = f"temp_images_vqa/{sample_id}.jpg"
        if not os.path.exists(image_path):
            image.save(image_path)

        #########################################################################

        print(f"\n========== Sample ID: {sample_id} ==========")

        for lang in languages:
            question_col = f"question_{lang}"
            answer_col = f"answer_{lang}"

            # Safety check if asnwer exists
            if question_col not in sample or answer_col not in sample:
                print(f"[{lang}] Missing data.")
                continue

            question = sample[question_col]
            gt_answer = sample[answer_col]

            #### Run inference using Gemma model ####

            messages = [
                {
                    "role": "system",
                    "content": [{"type": "text", "text": "You are a helpful assistant."}]
                },
                {
                    "role": "user",
                    "content": [
                        {"type": "image", "image": image_path},
                        {"type": "text", "text": question}
                    ]
                }
            ]

            inputs = processor.apply_chat_template(
                messages,
                add_generation_prompt=True,
                tokenize=True,
                return_dict=True,
                return_tensors="pt"
            ).to(model.device, dtype=torch_dtype)

            input_len = inputs["input_ids"].shape[-1]

            with torch.inference_mode():
                output = model.generate(**inputs, max_new_tokens=100, do_sample=False)
                pred_answer = processor.decode(output[0][input_len:], skip_special_tokens=True)

            ###########################################

            # Display result
            print(f"\n[{lang.upper()}]")
            print(f"Question: {question}")
            print(f"GT: {gt_answer}")
            print(f"Pred: {textwrap.fill(pred_answer, width=80)}")

        print("=" * 100)

    except Exception as e:
        print(f"Error processing sample {sample['sample_id']}: {e}")

    break  # remove this to run on the full dataset

config.json:   0%|          | 0.00/916 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/109k [00:00<?, ?B/s]

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

model-00004-of-00005.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

model-00005-of-00005.safetensors:   0%|          | 0.00/4.60G [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

Iterating samples:   0%|          | 0/9666 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:653: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `64` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(



========== Sample ID: 1 ==========

[EN]
Question: Is it overcast?
GT: No, it is clear.
Pred: No, it is not overcast. The sky in the image is a clear, bright blue.

[BN]
Question: এটা কি মেঘাচ্ছন্ন?
GT: না, এটা স্পষ্ট।
Pred: ছবিটি মেঘাচ্ছন্ন নয়। এটি একটি পরিষ্কার, নীল আকাশ দেখাচ্ছে। একজন ব্যক্তি একটি লাল
রঙের স্নোবোর্ড নিয়ে বাতাসে উড়ছেন।

[DE]
Question: Ist es bewölkt?
GT: Nein, es ist klar.
Pred: Nein, der Himmel ist blau und klar. Es scheint ein sonniger Tag zu sein.

[KO]
Question: 날이 흐린가요?
GT: 아니, 분명해요.
Pred: 아니요, 날씨가 맑습니다. 사진에서 하늘이 파랗게 보입니다.

[RU]
Question: Это пасмурная погода?
GT: Нет, это ясно.
Pred: Нет, погода на фотографии ясная и солнечная. Небо голубое, и нет признаков
облаков или пасмурности.


Iterating samples:   0%|          | 0/9666 [06:57<?, ?it/s]


[ZH]
Question: 是阴天吗？
GT: 不，很清楚。
Pred: No, the sky in the image is a clear, bright blue. It's a sunny day!
